<a href="https://colab.research.google.com/github/Mitchlai/course_3.0/blob/main/Langchain_AIAdb_Llama3_QA_Eval_0510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building AIA Chatbot by using retrieval augmented generation (RAG)

# Step1 : 安裝相關套件

In [41]:
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install chromadb

In [42]:
import os
import openai
import sys
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key  = os.environ['OPENAI_API_KEY']

#確認是否成功load api key
from dotenv import load_dotenv
load_dotenv()

False

In [43]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.llm import LLMChain
from langchain.schema import Document

# Step2 : 建立多種Emmbedings model

## 1、nomic-embed-text-v1.5

確保有跟PrimeHub vectorDB使用的模型一致：nomic-embed-text-v1.5

In [44]:
# install package
!pip install -U langchain-nomic

In [45]:
#nomic embdding 服務需要先申請 api key: https://atlas.nomic.ai
from langchain_nomic.embeddings import NomicEmbeddings
os.environ['NOMIC_API_KEY'] = 'nk-BCjzlgQXfYdZNA6kUZ-6Jeq1NIG2JhlQJaTe9BedpDc'
embedding = NomicEmbeddings(model="nomic-embed-text-v1.5")

In [46]:
#確認embedding model版本='nomic.embeddings
print(embedding)

In [47]:
#查看該文字轉換後的向量
# doc_result = embedding.embed_documents([text]) 查看完整多維度向量
text = "This is a test document."
query_result = embedding.embed_query(text)
query_result[:3] #確認文字已經可以轉為向量

[0.045318604, 0.038726807, -0.20007324]

# Step3 : 建立向量資料庫

## 連線至本地端PrimeHub vectorDB

In [48]:
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
# 連線設定
httpClient = chromadb.HttpClient(
  host='64.176.47.89', port=8000,
  settings=Settings(chroma_client_auth_provider="chromadb.auth.basic_authn.BasicAuthClientProvider",chroma_client_auth_credentials="admin:admin")
  )

# collections = httpClient.list_collections() #資料庫列表
#httpClient.delete_collection(name="xt131028") #刪除資料庫
#collection = httpClient.create_collection("xt131028") #可創建個人資料庫

# tell LangChain to use our client and collection name
db = Chroma(
    client=httpClient,
    collection_name="xt131028_v1",
    embedding_function=embedding,
)
# db.get() #查看所有資料

In [49]:
#查詢AIA本地端db
# query = "大型語言模型實作"
query = "技術領袖培訓全域班"
documents = db.similarity_search(query)
documents

[Document(page_content='常見問題: 申請退費\n回答: 親愛的學員你好，很遺憾你無法與我們一起為接下來的課程努力。\n退費的標準，我們會依照學員手冊退費辦法辦理。\n此外，請將學員證與發票退回至行政人員，也請告知匯款帳戶，以利我們作業。', metadata={'row': 12, 'source': 'QA_課中.csv'}),
 Document(page_content='RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10 日， 8 月 17 日 ， 8 月 24 日 每週六，共 4 週。上課方式實體課程，上課時間為 9:00 - 17:30 ( 提供午餐 ) ，4週共 30小時。上課地點台灣人工智慧學校台北總校 | 新板金融大樓地址：新北市板橋區中山路一段141號 14 樓適合對象具有 AI 基本知識的技術研發人員與主管 (具備 Python 程式能力)本課程涵蓋程式實作及模型微調，授課將運用 LangChain', metadata={'class_name': '大型語言模型實作進階班 (第四期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-tw/'}),
 Document(page_content='AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10', metadata={'class_name': '大型語言模型實作進階班 (第四期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-tw/'}),
 Document(page_content='日 週六 14:00

## 查詢向量資料庫的三種方式

In [50]:
#以字串查詢vectordb，回傳page_content內的資料
# question = "大型語言模型實作"
question = "技術領袖培訓全域班"
embedding_vector=embedding.embed_query(question)
research=db.similarity_search(question,k=3)
print(research[0].page_content)
print(research[1].page_content)
print(research[2].page_content)

常見問題: 申請退費
回答: 親愛的學員你好，很遺憾你無法與我們一起為接下來的課程努力。
退費的標準，我們會依照學員手冊退費辦法辦理。
此外，請將學員證與發票退回至行政人員，也請告知匯款帳戶，以利我們作業。
RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10 日， 8 月 17 日 ， 8 月 24 日 每週六，共 4 週。上課方式實體課程，上課時間為 9:00 - 17:30 ( 提供午餐 ) ，4週共 30小時。上課地點台灣人工智慧學校台北總校 | 新板金融大樓地址：新北市板橋區中山路一段141號 14 樓適合對象具有 AI 基本知識的技術研發人員與主管 (具備 Python 程式能力)本課程涵蓋程式實作及模型微調，授課將運用 LangChain
AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10


In [51]:
#將問題變為向量，以向量去查詢vectordb，回傳page_content內的資料
embedding_vector=embedding.embed_query(question)
research = db.similarity_search_by_vector(embedding_vector,k=3)
print(research[0].page_content)
print(research[1].page_content)
print(research[2].page_content)
# len(research)

常見問題: 申請退費
回答: 親愛的學員你好，很遺憾你無法與我們一起為接下來的課程努力。
退費的標準，我們會依照學員手冊退費辦法辦理。
此外，請將學員證與發票退回至行政人員，也請告知匯款帳戶，以利我們作業。
RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10 日， 8 月 17 日 ， 8 月 24 日 每週六，共 4 週。上課方式實體課程，上課時間為 9:00 - 17:30 ( 提供午餐 ) ，4週共 30小時。上課地點台灣人工智慧學校台北總校 | 新板金融大樓地址：新北市板橋區中山路一段141號 14 樓適合對象具有 AI 基本知識的技術研發人員與主管 (具備 Python 程式能力)本課程涵蓋程式實作及模型微調，授課將運用 LangChain
AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10


In [52]:
#以向量相似度（距離）去查詢vectordb，回傳page_content內的資料
research = db.similarity_search_with_score(question,k=3)
for research in research:
  print(research[0].page_content,research[1])

常見問題: 申請退費
回答: 親愛的學員你好，很遺憾你無法與我們一起為接下來的課程努力。
退費的標準，我們會依照學員手冊退費辦法辦理。
此外，請將學員證與發票退回至行政人員，也請告知匯款帳戶，以利我們作業。 0.5076097846031189
RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10 日， 8 月 17 日 ， 8 月 24 日 每週六，共 4 週。上課方式實體課程，上課時間為 9:00 - 17:30 ( 提供午餐 ) ，4週共 30小時。上課地點台灣人工智慧學校台北總校 | 新板金融大樓地址：新北市板橋區中山路一段141號 14 樓適合對象具有 AI 基本知識的技術研發人員與主管 (具備 Python 程式能力)本課程涵蓋程式實作及模型微調，授課將運用 LangChain 0.5420759916305542
AI 領域的重要性，因此特別開設此課程。本課程旨在讓學員不只理解大型語言模型的基本原理，更透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。我們期望，透過這樣的完整培訓，能為未來的 AI 領域孕育更多的專業人才，共同推進這一革命性技術的進步。課程成果圖：學員專題成果，地端檢索增強生成 (Retrieval-Augmented Generation, RAG) 串接資料庫進行資料檢索。課程資訊修業日期2024 年 8 月 03 日 ， 8 月 10 0.5507479906082153


# Step4 : 建立多個LLM Model進行比較


## 1、Call Groq_Llama3_8B

In [53]:
!pip install langchain-groq
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [54]:
llm_Groq_llama3 = ChatGroq(temperature=0, groq_api_key="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", model_name="llama3-8b-8192")
#MODEL_NAME = "llama3-70b-8192" or "llama3-8b-8192"
# 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
# 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

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm_Groq_llama3
chain.invoke({"text": "什麼是大型語言模型？使用正體中文回答"})

AIMessage(content='😊\n\n大型語言模型（Large Language Model，LLM）是一種使用深度學習技術訓練的語言模型，它可以處理和生成大量的自然語言文本。這種模型通常是基於神經網路架構，使用大量的文本資料進行訓練，以學習文本的語義、語法和語言特徵。\n\n大型語言模型的特點包括：\n\n1. 巨大規模：這些模型通常需要訓練數十億到數百億個參數，以捕捉文本的複雜性和多樣性。\n2. 自然語言處理：這些模型可以處理和生成自然語言文本，包括文本的生成、文本的理解和文本的翻譯等。\n3. 深度學習：這些模型使用深度學習技術，例如卷積神經網路（CNN）和循環神經網路（RNN），以學習文本的語義和語法。\n4. 高度自適性：這些模型可以根據文本的上下文和語境進行自適性處理，例如實現文本的生成和理解。\n\n大型語言模型的應用包括：\n\n1. 自然語言處理：這些模型可以應用於文本的生成、文本的理解和文本的翻譯等。\n2.  chatbot 和客服系統：這些模型可以應用於客服系統和chatbot中，以提供更好的客戶服務。\n3. 文本分析和挖掘：這些模型可以應用於文本分析和挖掘，以發掘文本中的隱藏信息和趨勢。\n\n總之，大型語言模型是一種強大的工具，可以幫助我們更好地理解和處理自然語言文本，並應用於各種領域中。', response_metadata={'token_usage': {'completion_time': 0.528, 'completion_tokens': 425, 'prompt_time': 0.022, 'prompt_tokens': 35, 'queue_time': None, 'total_time': 0.55, 'total_tokens': 460}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ff5d0c4-b2ca-4785-a998-fbe2c92d01a2-0')

## 2、Call PrimeHub_Llama3_8B

In [55]:
#透過langchain 建立後續要使用的 LLM model
!pip install -q langchain-community
!pip install -q langchain
!pip install -q requests
from langchain.llms import OpenAI
from langchain_community.llms import Ollama

In [56]:
#測試該url是否可以正常訪問
import requests

url = "https://75a8-140-109-17-42.ngrok-free.app" #http://3ece-140-109-17-42.ngrok-free.app/

try:
    requests.get(url)
except requests.exceptions.ConnectionError as e:
    print(f"Invalid URL: {e}")

In [57]:
llm_primehub_llama3 = Ollama(model="llama3", base_url="http://3ece-140-109-17-42.ngrok-free.app") #http://3ece-140-109-17-42.ngrok-free.app
llm_primehub_llama3.invoke("Tell me a joke")

'Why couldn\'t the bicycle stand up by itself?\n\nBecause it was two-tired!\n\n(Sorry, I know it\'s a bit of a "dad" joke, but I hope it brought a smile to your face!)'

# Step5 : 建立多種retriever進行比較

In [59]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n"
                + f"Content:\n{d.page_content}\n\n"
                + f"Metadata:\n{d.metadata}\n"
                for i, d in enumerate(docs)
            ]
        )
    )

In [60]:
!pip install --upgrade --quiet  lark langchain-chroma

## 1、Vector store-backed retriever



最基本的是Vector store-backed retriever ，使用向量存儲實現的搜索方法（如相似性搜索和MMR）來查詢向量存儲中的文字。参数都是与检索过程相关的

In [61]:

# 設定返回最相關的3個結果,啟用reduce_op_recursive模式,並啟用搜索質量反射
retriever = db.as_retriever(search_kwargs={"fetch_k": 10, #使用MMR 設定最多检索10个相关文档
                                           "k": 3, #並且返回3个最相关和最多样化的文档
                                           "mmr_score_cache": True, #启用MMR分数缓存
                                           "mmr_rerank_top_k": 30},#从相似度排序的前30个结果中选择候选文档进行MMR重新排序
                            retriever_mode="reduce_op_recursive",#启用reduce_op_recursive检索模式
                            #search_quality_reflection=True, #检索器在返回结果时会包含一个额外的search_quality_reflection字段,其中包含一个0到1之间的分数,表示检索质量的评估。分数越高,表示检索结果越能很好地回答查询。
                            # search_type 二擇一
                            # search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5} #以向量相似度（距離）去查詢vectordb，僅返回分數高於該閾值的文檔
                            search_type="mmr")     #使用MMR搜索策略

docs = retriever.invoke("大型語言模型實作進階班")
# len(docs)

pretty_print_docs(docs)

# print(docs[0].metadata)
# print(docs[1].metadata)
# print(docs[2].metadata)

Document 1:

Content:
大型語言模型實作初階班 (第三期)

Metadata:
{'class_name': '大型語言模型實作初階班 (第三期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llma-tw3/'}

----------------------------------------------------------------------------------------------------
Document 2:

Content:
大型語言模型實作進階班 (第二期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI

Metadata:
{'class_name': '大型語言模型實作進階班 (第二期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-2024-tw2/'}

----------------------------------------------------------------------------------------------------
Document 3:

Content:
大課程主題與方法：理解大型語言模型微調大型語言模型實作問答機器人大型語言模型部署上機實作熟悉大語言模型的技術能力。主題性的專題實作，培養專案實作的能力。課程效益能創建企業聊天機器人：設計和開發能夠理解並回應企業內部查詢的聊天機器人。實現機器人與公司數據庫的集成，以提供實時的查詢回應。進行機器人效能測試並根據反饋進行優化。能可微調開源大型語言模型：選擇適合的開源大型語言模型並進行微調，以符合公司特定的需求和應用場景。收集和整理適用於微調開源大型語言模型的業務相關數據集。評估微調後的大型語言模型效

Metad

## 2、Contextual compression retriever


檢索的一個挑戰是，通常您不知道將數據攝取到系統中時文檔存儲系統將面臨的特定查詢。這意味著與查詢最相關的資訊可能隱藏在包含大量不相關文本的文件中。通過應用程式傳遞完整文檔可能會導致更昂貴的LLM呼叫和更差的回應。

上下文壓縮旨在解決此問題。這個想法很簡單：您可以使用給定查詢的上下文壓縮它們，以便僅返回相關信息，而不是立即按原樣返回檢索到的文檔。這裡的「壓縮」是指壓縮單個文件的內容和過濾掉整個文件。




使用上下文壓縮器，以下LLMChainExtractor 和 LLMChainFilter 二擇一

In [62]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import LLMChainFilter

### 2-1 LLMChainExtractor

In [64]:
#選擇要使用的LLM
# llm = llm_primehub_llama3
llm = llm_Groq_llama3

#添加一個 LLMChainExtractor ，它將遍歷最初返回的文檔，並僅從每個文檔中提取與查詢相關的內容。
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever_1 = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever_1.invoke("摘要大型語言模型實作進階班的內容")
pretty_print_docs(compressed_docs)

Document 1:

Content:
Extracted relevant parts:

大型語言模型實作初階班 (第三期)

Metadata:
{'class_name': '大型語言模型實作初階班 (第三期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llma-tw3/'}

----------------------------------------------------------------------------------------------------
Document 2:

Content:
>>>
大型語言模型實作進階班 (第二期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI
>>>

Note: The extracted parts are the original text as-is, without any editing.

Metadata:
{'class_name': '大型語言模型實作進階班 (第二期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-2024-tw2/'}

----------------------------------------------------------------------------------------------------
Document 3:

Content:
>>>
大課程主題與方法：理解大型語言模型微調大型語言模型實作問答機器人大型語言模型部署上機實作熟悉大語言模型的技術能力。
主題性的專題實作，培養專案實作的能力。
課程效益能創建企業聊天機器人：設計和開發能夠理解並回應企業內部查詢的聊天機器人。
實現機器人與公司數據庫的集成，以提供實時的

### 2-2 LLMChainFilter

In [65]:
#選擇要使用的LLM
# llm = llm_primehub_llama3
llm = llm_Groq_llama3

#添加一個 LLMChainFilter，該壓縮器稍微簡單一些，但更強大，它使用LLM鏈來決定要過濾掉哪些最初檢索的文檔以及要返回哪些文檔，而無需操作文檔內容。
_filter = LLMChainFilter.from_llm(llm)
compression_retriever_2 = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever_2.invoke("摘要大型語言模型實作進階班的內容")
pretty_print_docs(compressed_docs)

Document 1:

Content:
大型語言模型實作初階班 (第三期)

Metadata:
{'class_name': '大型語言模型實作初階班 (第三期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llma-tw3/'}

----------------------------------------------------------------------------------------------------
Document 2:

Content:
大型語言模型實作進階班 (第二期) 招生簡章🚀透過上機實作教學和專題實作導引，熟練地將這些技術應用於實際問題中，開發出具有價值的 AI 應用。💪👍🤝課程大綱立即報名課程簡介基於上百家企業需求想要用大型語言模型建置企業內部的「企業大腦」，因各企業的資料有機敏性與獨特性，無法使用公開的大型語言模型。此外微調及部署大型語言模型服務有一定的門檻在，不論是在設備上、資料處理上、技術上等。台灣人工智慧學校深知大型語言模型在當今 AI

Metadata:
{'class_name': '大型語言模型實作進階班 (第二期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-2024-tw2/'}

----------------------------------------------------------------------------------------------------
Document 3:

Content:
大課程主題與方法：理解大型語言模型微調大型語言模型實作問答機器人大型語言模型部署上機實作熟悉大語言模型的技術能力。主題性的專題實作，培養專案實作的能力。課程效益能創建企業聊天機器人：設計和開發能夠理解並回應企業內部查詢的聊天機器人。實現機器人與公司數據庫的集成，以提供實時的查詢回應。進行機器人效能測試並根據反饋進行優化。能可微調開源大型語言模型：選擇適合的開源大型語言模型並進行微調，以符合公司特定的需求和應用場景。收集和整理適用於微調開源大型語言模型的業務相關數據集。評估微調後的大型語言模型效

Metad

## 3、MultiQueryRetriever
using an LLM to generate multiple queries from different perspectives for a given user input query.

通過 MultiQueryRetriever 給定的使用者輸入查詢，由LLM從不同角度生成多個查詢，自動執行提示調整過程。對於每個查詢，它都會檢索一組相關文檔，並在所有查詢中採用唯一的聯合，以獲得更大的潛在相關文檔集。通過對同一問題生成多個視角， MultiQueryRetriever 可能能夠克服基於距離的檢索的一些局限性，並獲得更豐富的結果集。

https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/

### 3-1 MultiQuery_retriever_from_llm_1

In [97]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "What are the approaches to Task Decomposition?"
llm = llm_Groq_llama3
MultiQuery_retriever_from_llm_1 = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

In [98]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [99]:
unique_docs = MultiQuery_retriever_from_llm_1.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three alternative versions of the original question:', '', 'What are the strategies for breaking down complex tasks into manageable sub-tasks?', '', 'What are the methods for decomposing large tasks into smaller, more manageable components?', '', 'How do experts approach the process of decomposing complex tasks into smaller, more manageable tasks?', '', 'These alternative questions aim to capture different aspects of the original question, such as the focus on strategies, methods, and expert approaches. By generating multiple perspectives on the user question, we can increase the chances of retrieving relevant documents from the vector database that may not have been retrieved by the original question alone.']


12

### 3-2 MultiQuery_retriever_from_llm_2

提供您自己的提示

In [100]:
from typing import List
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# # Output parser will split the LLM result into a list of queries
# class LineList(BaseModel):
#     # "lines" is the key (attribute name) of the parsed output
#     lines: List[str] = Field(description="Lines of text")


# class LineListOutputParser(JsonOutputParser):
#     def __init__(self) -> None:
#         super().__init__(pydantic_object=LineList)

#     def parse(self, text: str) -> LineList:
#         lines = text.strip().split("\n")
#         return LineList(lines=lines)


# output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines. Please answer in Chinese.
    Original question: {question}""",
)

llm = llm_Groq_llama3

# Chain
# llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT) #使用llm_chain會跑很慢

# Other inputs
question = "摘要大型語言模型實作進階班的內容"

In [101]:
# Run
MultiQuery_retriever_from_llm_2 = MultiQueryRetriever(
    retriever=db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
# unique_docs = MultiQuery_retriever_from_llm.invoke(query="摘要大型語言模型實作進階班的內容")
unique_docs = MultiQuery_retriever_from_llm_2.invoke("摘要大型語言模型實作進階班的內容")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: Here are three alternative versions of the original question:

摘要大型語言模型實作進階班的內容

大型語言模型實作進階班的摘要內容

進階班大型語言模型實作的摘要內容

These alternative questions aim to capture different aspects of the original question, such as:

* Emphasizing the summary aspect of the content
* Swapping the order of the words to create a different perspective
* Focusing on the advanced nature of the course

By generating these alternative questions, we can increase the chances of retrieving relevant documents from the vector database that may not have been retrieved by the original question alone.


58

### other

https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

# Step6 : 建立 PromptTemplate


In [102]:
# 建立 PromptTemplate
from langchain.prompts import PromptTemplate

# template = """You are a customer support agent. You are designed to be as helpful as possible while providing only factual information. You should be friendly, but not overly chatty. Context information is below. Given the context information and not prior knowledge, answer the query. Always say "thanks for asking!" at the end of the answer.
# template = """Context information is below. Given the context information and not prior knowledge. Please answer the question by zh-tw language If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
template = """Use the following pieces of context to answer the question at the end. Please answer in Chinese. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Step7 : 建立問答鏈 RetrievalQA

In [115]:
from langchain.chains import RetrievalQA

# question = "大型語言模型實作進階班的開課日期是什麼時候？"
# question = "介紹技術領袖培訓全域班第五期的課程內容和修業期間"
# question = "介紹大型語言模型實作"
# question = "介紹經理人週末研修班課程"
# question = "我已完成報名繳費,何時可以收到發票?"
# question = "我要如何申請退費?"
question="摘要大型語言模型實作進階班的內容"

qa_chain = RetrievalQA.from_chain_type(llm=llm_Groq_llama3, #選擇要使用的 llm_Groq_llama3、llm_primehub_llama3
                                       retriever=MultiQuery_retriever_from_llm_2, # 多擇一 retriever or compression_retriever_1 or compression_retriever_2 or MultiQuery_retriever_from_llm_1 or MultiQuery_retriever_from_llm_2
                                       return_source_documents=True,
                                       chain_type="stuff", #map_reduce
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

result = qa_chain({"query": question})
result["result"] #查看結果

INFO:langchain.retrievers.multi_query:Generated queries: Here are three alternative versions of the original question:

摘要大型語言模型實作進階班的內容

大型語言模型實作進階班的摘要內容

進階班大型語言模型實作的摘要內容

These alternative questions aim to capture different aspects of the original question, such as:

* Emphasizing the summary aspect of the content
* Swapping the order of the words to create a different perspective
* Focusing on the advanced nature of the course

By generating these alternative questions, we can increase the chances of retrieving relevant documents from the vector database that may not have been retrieved by the original question alone.


'Here is a summary of the content of the Large Language Model Implementation Advanced Course (LLM-B):\n\n**Course Overview**\n\n* The course is designed for those who want to learn how to implement large language models in their work.\n* The course covers the basics of large language models, including their applications and limitations.\n* The course also covers the implementation of large language models using Python and PyTorch.\n\n**Course Outline**\n\n* Week 1: Introduction to Large Language Models\n* Week 2: Implementation of Large Language Models using Python and PyTorch\n* Week 3: Applications of Large Language Models\n* Week 4: Advanced Topics in Large Language Models\n\n**Course Objectives**\n\n* To understand the basics of large language models\n* To learn how to implement large language models using Python and PyTorch\n* To understand the applications and limitations of large language models\n* To learn how to use large language models in real-world applications\n\n**Course 

retriever ANS : 本課程主要涵蓋大型語言模型的設計、優化和應用，透過實作和專題實作導引，學習如何將大型語言模型應用於實際問題中。課程內容包括大型語言模型的運作原理、微調和部署等。 thanks for asking!

compression_retriever_1 ANS : 大型語言模型實作進階班的內容包括：理解大型語言模型微調、實作問答機器人、部署上機實作、熟悉大語言模型的技術能力、主題性的專題實作、企業聊天機器人設計和開發、機器人與公司數據庫的集成、機器人效能測試、微調開源大型語言模型等。 thanks for asking!

compression_retriever_2 ANS : 本課程主要內容是大型語言模型的微調和部署，學習如何將大型語言模型應用於實際問題中，開發具有價值的 AI 應用。課程中還涵蓋了機器人設計和開發、集成公司數據庫、機器人效能測試和優化等內容。課程的目的是培養學生的專案實作能力和技術能力。
Thanks for asking!

MultiQuery_retriever_from_llm_1 ANS : 本課程涵蓋大型語言模型實作進階班的內容，包括 LangChain 課程、微調大型語言模型、實作進階班等。
thanks for asking!

MultiQuery_retriever_from_llm_2 ANS : By generating these alternative questions, we can increase the chances of retrieving relevant documents from the vector database that may not have been retrieved by the original question alone.
Here is a summary of the content of the Large Language Model Implementation Advanced Course (LLM-B):\n\n**Course Overview**\n\n* The course is designed for those who want to learn how to implement large language models in their work.\n* The course covers the basics of large language models, including their applications and limitations.\n* The course also covers the implementation of large language models using Python and PyTorch.\n\n**Course Outline**\n\n* Week 1: Introduction to Large Language Models\n* Week 2: Implementation of Large Language Models using Python and PyTorch\n* Week 3: Applications of Large Language Models\n* Week 4: Advanced Topics in Large Language Models\n\n**Course Objectives**\n\n* To understand the basics of large language models\n* To learn how to implement large language models using Python and PyTorch\n* To understand the applications and limitations of large language models\n* To learn how to use large language models in real-world applications\n\n**Course P

In [106]:
result["source_documents"] #查看來源檔案

[Document(page_content='are few-shot learnersTraining language models to follow instructions with human feedbackLLaMA: Open and Efficient Foundation Language ModelsLlama 2: Open Foundation and Fine-Tuned Chat', metadata={'class_name': '大型語言模型實作進階班 (LLM-B) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-tw1/'}),
 Document(page_content='few-shotlearnersTraining language models tofollow instructions with human feedbackLLaMA: Open and Efficient Foundation Language ModelsLlama 2: Open Foundation andFine-Tuned Chat', metadata={'class_name': '大型語言模型實作進階班 (第二期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-llmb-2024-tw2/'}),
 Document(page_content='(3 小時)LLM 簡介 (3 小時)：專門領域教授上機習作 (14 小時)文字資料 (4 小時)：助教/AIA 工程師微調大型語言模型 (7 小時)：助教/AIA 工程師問答機器人 (3 小時)：助教/AIA 工程師部署大型語言模型(1 小時)：助教/AIA 工程師專題實作 (4 週，時數 5 小時)專題實作引導 (4 小時)：助教/AIA 工程師專題成果分享 (1 小時)：助教/AIA 工程師專題演講 (8', metadata={'class_name': '大型語言模型實作進階班 (第二期) 招生簡章', 'source': '簡章', 'url': 'https://aiacademy.tw/admission-l

## 其他待測試

In [ ]:

# # 假设您有一个可以转换为检索器的数据库对象
# try:
#     # 使用配置好的 llm 创建 QA 链
#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm,
#         retriever=db.as_retriever(),
#         return_source_documents=True,
#         chain_type="stuff",
#         chain_type_kwargs={"prompt": "请根据您的需求定制提示"}
#     )

#     question = "摘要技術領袖培訓全域班課程大綱"
#     result = qa_chain({"query": question})  # 把question丢入RetrievalQA
#     print(result["result"])  # 查看结果

# except Exception as e:
#     print(f"执行过程中发生错误: {e}")


执行过程中发生错误: name 'RetrievalQA' is not defined


In [ ]:
# if "search_quality_reflection" in result:
#     print(result["search_quality_reflection"])
# else:
#     print("Search quality reflection is not available for this query.")

#通过print(result["search_quality_reflection"])输出并查看这个分数。
#分数范围是0到1之间的浮点数,越接近1表示检索质量越好,能更好地回答查询。如果分数较低,你可以考虑采取其他措施,比如获取更多相关文档、提示用户重新表达查询等。

NameError: name 'result' is not defined

# Step8 : 建立本機evaluation

## LLM-Generated examples

In [ ]:
#利用 LLM 自動化生成 (自動從每個文檔建立一個 Q, A pair
from langchain.evaluation.qa import QAGenerateChain

In [ ]:
data = db.get()['documents'] # 取原始文本 chunks
print(type(data))
print(data)

<class 'list'>
['常見問題: 我的學員證不見了，要申請，要怎麼給你們錢?\n回答: 您好，請發信至hi@aiacademy.tw 申請，以便我們作業紀錄登記；若您確定申請補發學員證，我們將製作好之後，會現場給您學員證+發票，屆時會與您收500元。', '常見問題: 詢問師資 （醫療專班）\n回答: 台灣人工智慧學校提供專業的師資與課程，也安排醫界的先進前來授課，分享智慧醫療應用與案例的分享。\n希望能透過五週的課程，讓工作繁忙的醫師及護理相關人員，能快速、系統化掌握人工智慧技術的最新趨勢與發展應用能力，同時進行醫療實作案例分享。\n學校將邀請中央研究院等學研單位及產業界，擁有豐富經驗與理論背景的講師，進行機器學習、深度學習、智慧生醫等重要的人工智慧核心技術與應用課程講授。\n預計將培養上千位醫療產業從業人員，擁有定義問題、解決及場域導入方式等關鍵能力，並能實際應用在醫療工作上為全台醫療產業注入維持競爭力的能量。\n本次北部醫療專班的師資\n陳弘軒        國立中央大學 資工系副教授        機器學習\n蔡炎龍        國立政治大學應用數學系副教授        深度學習\n莊永裕        台大資工系 教授        電腦視覺\n蔣榮先         成大資工系教授暨醫院健康數據中心執行長        智慧醫療應用', '18,000 元，團報達 10 人以上每人新台幣 16,000 元。大量團報請另洽專員：02-85123731 #12上課地點：桃園龜山文化一路 15號 (林口長庚醫院 研究大樓一樓綜合會議廳)報名方式：本招生採網路報名，請於報名截止日 2023/07/10前（含 07 月 10', '500 元，其餘費用全數退還；活動已開始，部分取消則依取消天數比例退還。注意事項因課程需要，請學員必須自備筆電來參加本營隊。錄取學員如經發現報名資格不符規定，或所繳資料有偽造、變造、假借、冒用、剽竊、內容不實、塗改等情事，取消資格，並應負法律責任，且不發給任何有關營隊之證書，如係在營期後始發覺者，將撤銷其營隊證書。主辦單位保有所有相關規定最終解釋權及活動修改、變更之權利，若有相關異動將會公告於網站，', 'Google 前董事總經理 Appier & ikala 董事1991畢業於台大資工所博士班1993年加入中研院資訊

In [ ]:
#選擇要使用的LLM
# llm = llm_primehub_llama3
llm = llm_Groq_llama3

# 自動從每個文檔的每個 chunks 建立一個 QA pair
example_gen_chain = QAGenerateChain.from_llm(llm)

# examples = example_gen_chain.apply(
#     [{"doc": "請用中文: " + t} for t in data[:50]] # 先產 50 個看看 (實測僅 OpenAI model 可轉成中文問答生成，llama3-70b-8192 生成的依然是英文)
# )

examples = example_gen_chain.apply_and_parse(
    [{"doc": "請用中文: " + t} for t in data[:50]] # 先產 50 個看看 (實測僅 OpenAI model 可轉成中文問答生成，llama3-70b-8192 生成的依然是英文)
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [ ]:
examples[0]

{'qa_pairs': {'query': 'What is the recommended method for requesting a replacement student ID card if it is lost?',
  'answer': 'Sending an email to hi@aiacademy.tw with a request for a replacement student ID card.'}}

In [ ]:
print(type(examples[0]))
examples[0]['qa_pairs']

<class 'dict'>


{'query': 'What is the recommended method for requesting a replacement student ID card if it is lost?',
 'answer': 'Sending an email to hi@aiacademy.tw with a request for a replacement student ID card.'}

In [ ]:
# 因為建立的 examples 會是兩層的 dictionary list, 只組合第二層的 dictionary 作為最後的 qa_examples list
qa_examples = []
for example in examples:
  qa_examples.append(example['qa_pairs'])
print(qa_examples)
print(len(qa_examples))

[{'query': 'What is the recommended method for requesting a replacement student ID card if it is lost?', 'answer': 'Sending an email to hi@aiacademy.tw with a request for a replacement student ID card.'}, {'query': 'Who are the professors invited to teach the core technologies and applications of artificial intelligence in the medical field?', 'answer': 'The professors invited include Chen Hong-xuan, Cai Yan-long, Zhuang Yong-yu, and Jiang Rong-xian.'}, {'query': 'What is the price for each person if there are 10 or more people in a group?', 'answer': 'NT$16,000'}, {'query': 'What will happen to the fees if a participant cancels their registration after the activity has started?', 'answer': 'The fees will be refunded proportionally based on the cancellation date.'}, {'query': 'Who was the director-general of Google?', 'answer': 'Eric Tsai'}, {'query': 'What will happen to students who have submitted false or forged documents, such as fake academic records or certificates?', 'answer': '

## LLM assisted evaluation

In [ ]:
predictions = qa_chain.apply(qa_examples) # 對所有 QA 範例進行預測，以供後續評估

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use batch instead.
  warn_deprecated(


In [ ]:
print(len(predictions)) # 確認數量

50


### QAEvalChain

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
# 對所有 QA 範例和預測結果進行比對評估，返回一堆評估分級
graded_outputs = eval_chain.evaluate(qa_examples, predictions)
len(graded_outputs)

50

In [ ]:
graded_outputs[0]

{'results': 'GRADE: INCORRECT'}

In [ ]:
for i, eg in enumerate(qa_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted " + graded_outputs[i]['results'])
    print()

Example 0:
Question: What is the recommended method for requesting a replacement student ID card if it is lost?
Real Answer: Sending an email to hi@aiacademy.tw with a request for a replacement student ID card.
Predicted Answer: 沒有找到相關信息，無法回答這個問題。 thanks for asking!
Predicted GRADE: INCORRECT

Example 1:
Question: Who are the professors invited to teach the core technologies and applications of artificial intelligence in the medical field?
Real Answer: The professors invited include Chen Hong-xuan, Cai Yan-long, Zhuang Yong-yu, and Jiang Rong-xian.
Predicted Answer: 不知道（wǒ bù zhī dào）. Thanks for asking!
Predicted QUESTION: Who are the professors invited to teach the core technologies and applications of artificial intelligence in the medical field?
STUDENT ANSWER: 不知道（wǒ bù zhī dào）. Thanks for asking!
TRUE ANSWER: The professors invited include Chen Hong-xuan, Cai Yan-long, Zhuang Yong-yu, and Jiang Rong-xian.
GRADE: INCORRECT

Example 2:
Question: What is the price for each person i

### Scoring Evaluator

In [ ]:
from langchain.evaluation import load_evaluator
evaluator = load_evaluator("labeled_score_string", llm=llm)

In [ ]:
# Sample test
eval_result = evaluator.evaluate_strings(
    prediction="You can find them in the dresser's third drawer.",
    reference="The socks are in the third drawer in the dresser",
    input="Where are my socks?",
)
print(eval_result['score'])

8


In [ ]:
# 對所有 QA 範例和預測結果進行比對評估，返回一堆評估分級
# graded_outputs = eval_chain.evaluate(qa_examples, predictions)
# len(graded_outputs)

for i, eg in enumerate(qa_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Score: " +
          str(
              evaluator.evaluate_strings(
                  prediction=predictions[i]['result'],
                  reference=predictions[i]['answer'],
                  input=predictions[i]['query']
                  )['score']
              )
          )
    print()

# eval_result =
# print(eval_result)

Example 0:
Question: What is the recommended method for requesting a replacement student ID card if it is lost?
Real Answer: Sending an email to hi@aiacademy.tw with a request for a replacement student ID card.
Predicted Answer: 沒有找到相關信息，無法回答這個問題。 thanks for asking!
Predicted Score: 2

Example 1:
Question: Who are the professors invited to teach the core technologies and applications of artificial intelligence in the medical field?
Real Answer: The professors invited include Chen Hong-xuan, Cai Yan-long, Zhuang Yong-yu, and Jiang Rong-xian.
Predicted Answer: 不知道（wǒ bù zhī dào）. Thanks for asking!
Predicted Score: 1

Example 2:
Question: What is the price for each person if there are 10 or more people in a group?
Real Answer: NT$16,000
Predicted Answer: 團體報名專屬優惠：報名人數達 5 位以上，每人優惠價為新台幣 32,000。thanks for asking!
Predicted Score: 2

Example 3:
Question: What will happen to the fees if a participant cancels their registration after the activity has started?
Real Answer: The fees will be refu

# Step9 : 建立LangSmith evaluation

intsall LangSmith

In [ ]:
!pip install -U langsmith # evaluation platform

其他環境設定

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true" # 告訴 LangChain 啟用記錄追蹤 (如使用 RunTree API，可以不用設定啟用)
# os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}" # 指定記錄到哪個 project (預設記錄在 default project；指定 project 不存在時，會自動協助建立；詳緦可登入 LangSmith 平台查看)
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
os.environ['LANGCHAIN_API_KEY'] = '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'

## 建立 Langsmith 客戶端

In [ ]:
from langsmith import Client

client = Client() # 建立 langsmith 客戶端以與 API 互動

# 指定 LangSmith Dataset (如未建立，需執行下段「建立 dataset 及 QA samples」程式)

dataset_name = "AIA Q&A Dataset" # 完整 AIA Q&A 共 47 筆
# dataset_name = "AIA Q&A Dataset_oneTest" # 只有一筆 QA

## 建立 dataset 及 QA samples(婷方已建立 可略過）
在 LangSmith 平台上建立新的資料集和樣本資料


In [ ]:
# Define dataset:
# dataset_aia = client.create_dataset(dataset_name, description="## AIA Q&A集: https://aiacademy.tw/AIA_QA/qa_data_12AJAJCIqeck45.php (僅供專題使用)")

In [ ]:
# # 建立樣本資料 - AIA Q&A 集
# import requests

# # 載入 AIA Q&A 集
# response = requests.get("https://aiacademy.tw/AIA_QA/qa_data_12AJAJCIqeck45.php")
# data = response.json()  # Decode the JSON data
# queries = []
# answers = []
# for i in data:
#   if len(i)<2 or i[0]=="常見問題": #處理標題/分類 列
#     pass
#   else:
#     query = {"query":i[0]}
#     answer = {"answer":i[1]}
#     queries.append(query)
#     answers.append(answer)

# # 上傳至 LangSmith "AIA Q&A Dataset" 資料集
# client.create_examples(
#     inputs = queries,
#     outputs = answers,
#     dataset_id = dataset_aia.id
# )

## 建立evaluation使用的PromptTemplate

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
Respond with CORRECT or INCORRECT:
Grade:
"""

PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)

qa_evaluator = LangChainStringEvaluator("qa", config={"llm": llm, "prompt": PROMPT})

評估 llm 回答的長度，確保我們的答案 (如預期的) 簡短明瞭

In [ ]:
# 檢查實際輸出是否小於預期結果長度的 2 倍
from langsmith.schemas import Run, Example

def evaluate_length(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("answer") or ""
    score = int(len(prediction) < 2 * len(required))
    return {"key":"length", "score": score}

## 針對question進行Answer的prediction

In [ ]:
def llm_prediction(question):
  result = qa_chain.invoke({"query": question})
  return result["result"]

In [ ]:
# 定義一個簡單的包裝器，將 (LangSmith) 資料集中的 input key 對應到我們想要呼叫的函數，然後也將函數的輸出對應到我們期望的 output key。
def langsmith_app(inputs):
    output = llm_prediction(inputs["query"])
    return {"answer": output}

In [ ]:
#執行後，可至下方介面查看結果
# AIA Q&A Dataset:
# https://smith.langchain.com/public/6cd4801e-f174-4674-ba06-dda2ab18191d/d
# AIA Q&A Dataset_oneTest:
# https://smith.langchain.com/public/eb428b1b-b27e-47a4-8111-dfa612ddb8ef/d
# from langsmith.evaluation import evaluate

experiment_results = evaluate(
    langsmith_app, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[evaluate_length, qa_evaluator], # The evaluators to score the results
    experiment_prefix="131027_Ollama-llama3_DB-xt131028_v1-Nomic-embedding", # A prefix for your experiment names to easily identify them
)

View the evaluation results for experiment: '131027_Ollama-llama3_DB-xt131028_v1-Nomic-embedding-651ff6ac' at:
https://smith.langchain.com/o/6ee61aca-f04a-5cdf-9665-bb8fceaeb496/datasets/d657a3ef-050f-45a6-ac90-d3cc4877c030/compare?selectedSessions=b260b011-d8e9-4d13-b262-944d95b68265




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01hxghnbqnfmws4ygz7gy1k9aq` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.626s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'requests', 'code': 'rate_limit_exceeded'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01hxghnbqnfmws4ygz7gy1k9aq` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.535s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'requests', 'code': 'rate_limit_exceeded'}}
ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization

# 附件：AIA FAQ 測試資料

In [ ]:
# 註冊繳費
question: 我已完成報名繳費,何時可以收到發票?
answer: 個人用電子發票會於開學後第二週上課日由學務通知領取。公司報帳用電子發票會於開學後第二週由學務 mail 至學員報名時的電子信箱。若需提早或延後開立發票,請發信至hi@aiacademy.tw詢問。

question: 我已過繳費期限?
answer: 因原先繳費期限已過期,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們將會重新寄發{報名及註冊的登記已可進行下一步報名作業}信件至您的電子信箱。

question: 註冊後,我選擇臨櫃匯款,該匯到哪裡?
answer: 選擇臨櫃匯款者,請匯款至 : 玉山銀行 南港分行 金融機構代碼：808 1182 戶名：財團法人台灣人工智慧學校基金會 帳號：1182-940-016585。注意 : 選擇臨櫃匯款繳費的學員請務必於匯款完成後來信至 hi@aiacademy.tw 提供您的姓名、報名課別、匯款單或匯款帳號後五碼,以利我們對帳後更新你的註冊狀態。

question: 如何繳費?
answer: 您好,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們會於系統更新你的審核狀態並重新寄發{報名及註冊的登記已可進行下一步報名作業}至您的電子信箱。若您報名的是經理人週末研修班或技術領袖全域班獲錄取者,請於收到錄取通知後 3 天內完成註冊繳費。繳費方式可選擇線上金流 (信用卡) 或臨櫃匯款。若選擇臨櫃匯款者,請匯款至 : 玉山銀行 南港分行 金融機構代碼：808 1182 戶名：財團法人台灣人工智慧學校基金會 帳號：1182-940-016585。\n注意 : 選擇臨櫃匯款繳費的學員請務必於匯款完成後來信至 hi@aiacademy.tw 提供匯款單或匯款帳號後五碼,以利我們對帳後更新你的註冊狀態。

# 退費
question: 我要如何申請退費?
answer: 很遺憾您無法與我們一起為接下來的課程努力。\請您發信至hi@aiacademy.tw 提供您的班別跟姓名。 退費的標準,我們會依照學員手冊退費辦法辦理,後續也請提供你您帳號(含分行)資訊(存摺影本),以利我們後續匯款。謝謝

question: 申請退費
answer: 親愛的學員你好,很遺憾你無法與我們一起為接下來的課程努力。退費的標準,我們會依照學員手冊退費辦法辦理。此外,請將學員證與發票退回至行政人員,也請告知匯款帳戶,以利我們作業。

# 請假
question: 如何請假?
answer: 若學員因故有請假需求,請填妥請假表單進行申請 https://reurl.cc/xZNOAL校務行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。請假申請審核過後,仍計入缺席時數。本課程無補課機制。

#報名
question: 我的報名已過期?
answer: 因原先報名連結已過期,請發信至 hi@aiacademy.tw 並提供您的姓名及報名時填寫的電子郵件信箱。我們將會重新寄發{報名及註冊的登記已可進行下一步報名作業}信件至您的電子信箱。

question: 我是文科的人,我可以報名你們的課程嗎？
answer: 我們非常歡迎不同產業的人進來,希望可以推動不同產業專業的人,能認識AI這些知識。

#學費補助
question: 請問你們有學費補助嗎？因為我看我們醫院有補助計畫,那如果你們這一期沒有,我是否可以轉下一期？
answer: 我們學校沒有提供學費補助,應該是貴醫院單位的補助計畫,可能需要您與貴單位做內部確認,是否能申請院內補助,我本校無權涉入。

#電腦
question: 課程是否需要帶電腦?
answer: 課程當中並沒有需要使用到電腦的部(除了工程技術類課程外),您可以依據自身上課需要攜帶筆電。

#午餐
question: 請問你們所有課程都有提供午餐嗎？
answer: 因為我們的班別有經理人周末研修班、技術領袖全域班、智慧醫療專班和產業專班,目前只有經理人周末研修班/智慧醫療專班和產業專班有提供午餐。

#結業
question: 學員要通過那些門檻才會有結業證書？是否有課綱介紹？
answer: 結業門檻等相關行政細節資訊列於學員手冊上,待開學當周會寄發給各位學員。課綱可以先參考官網 https://aiacademy.tw/

#課程差別
question: 醫療專班跟技術班還有經理人班的差別是？
answer: 醫療專班：智慧醫療專班是台灣人工智慧學校為台灣醫療產業從業人員所設計的密集課程,期待透過智慧醫療專班,讓醫療產業從業人員能夠快速、系統化掌握人工智慧技術的最新趨勢,並徹底瞭解人工智慧如何運作,以及它的能力、侷限及未來發展,並能實際應用在醫療工作的每個場景。\n智慧金融專班:\n給予專班學員金融業全面性應用到的人工智慧知識,再透過實務議題、案例分享與場域實地參訪的印證,即時掌握智慧金融應用的趨勢。同時安排學員們進行深入的專題討論,將課堂傳授的知識運用於專題及實作,領略這些技術的實際應用方式及限制,以及進行學員間深入的交流學習。\n經理人班：\n經理人週末研修班是台灣人工智慧學校在技術領袖培訓班之後,第二個專門為台灣目前產業需求所設計的班別。我們預期透過經理人週末研修班,讓各產業的經理人能有一個場域,與不同產業的經理人一同學習及理解人工智慧的技術概觀,對於人工智慧技術能有全面性的大局觀,而且徹底瞭解人工智慧是如何運作的,以及它的能力、侷限及未來發展,才能讓產業欲以人工智慧進行產業升級時,能有清楚的方向感來帶領企業前進。\n技術班：\n技術領袖培訓班期為台灣育成具有人工智慧技術思維與實戰經驗的技術領袖人才,讓產業欲以人工智慧進行產業升級時,不再為缺乏人才所束縛。

#師資
question: 詢問智慧醫療專班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排醫界的先進前來授課,分享智慧醫療應用與案例的分享。希望能透過五週的課程,讓工作繁忙的醫師及護理相關人員,能快速、系統化掌握人工智慧技術的最新趨勢與發展應用能力,同時進行醫療實作案例分享。\n學校將邀請中央研究院等學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、智慧生醫等重要的人工智慧核心技術與應用課程講授。\n預計將培養上千位醫療產業從業人員,擁有定義問題、解決及場域導入方式等關鍵能力,並能實際應用在醫療工作上為全台醫療產業注入維持競爭力的能量。\n本次北部醫療專班的師資\n陳弘軒        國立中央大學 資工系副教授        機器學習\n蔡炎龍        國立政治大學應用數學系副教授        深度學習\n莊永裕        台大資工系 教授        電腦視覺\n蔣榮先         成大資工系教授暨醫院健康數據中心執行長        智慧醫療應用\n及其他醫界先進們,目前講師還在安排調整,官網只會放上課程大綱,待開課前會寄出學員手冊,裡面的課程表會有當期課程及講師,屆時再提供您參考\n

question: 詢問智慧製造專班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排了在製造業有深厚實務經驗的先進前來授課,分享智慧製造應用與案例的分享。希望能透過二天共七週的課程,讓來參與智慧製造課程的各位,能充份吸收並掌握人工智慧技術的最新趨勢與發展應用能力。\n學校將邀請中央研究院等學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、智慧製造等重要的人工智慧核心技術與應用課程講授。\n本次產業專班的師資\n吳漢銘        國立政治大學統計學系副教授      資料科學與大數據分析\n陳弘軒       國立中央大學 資工系副教授        機器學習與演算法概論\n許懷中       逢甲大學資訊工程學系副教授     深度學習 \n莊永裕       台大資工系 教授                          電腦視覺 \n李家岩       國立臺灣大學資訊管理學系教授  智慧製造與生產線上的資料科學\n智慧製造實務案例分享及應用,目前講師還在安排調整,官網只會放上課程大綱,待開課前會寄出學員手冊,裡面的課程表會有當期課程講義及授課講師,屆時再提供您參考

question: 詢問經理人班師資
answer: 台灣人工智慧學校提供專業的師資與課程,也安排有深厚實務經驗的先進前來授課,分享人工智慧的應用與案例的分享。希望能透過14週的課程,從基礎核心、實作學習到產業應用,讓來參與課程的學員,充份吸收並掌握人工智慧技術的最新趨勢與發展應用能力。   學校將邀請學研單位及產業界,擁有豐富經驗與理論背景的講師,進行機器學習、深度學習、電腦視覺、資料探勘等重要的人工智慧核心技術與應用課程講授。 本次經理人班的師資 吳漢銘 國立政治大學統計學系副教授 資料科學與大數據分析 陳弘軒 國立中央大學 資工系副教授 機器學習與演算法概論 蔡炎龍 國立政治大學應用數學系副教授 深度學習 莊永裕 國立臺灣大學資訊工程學教授 電腦視覺 陳宜欣 國立清華大學資訊工程學系教授 資料探勘 （學校保有修改、變更課程內容之權利）\n以及業界先進產業案例的分享及應用。\n官網只會放上課程大綱,待開課前會寄出學員手冊,登入學員專區裡面的課程表,會有當期課程講義及授課講師。\n本次經理人班也加入AIGC實戰應用,學會使用相關工具如：ChatGPT、 Midjourney 等來生成文字與圖像,結合情境應用,有效減輕工作負擔,瞬間提升工作效率。

#發票
question: 學員發票什麼時候拿
answer: 個人用電子發票會於開學後第二週上課日由學務通知領取。\r公司報帳用電子發票會於開學後第二週由學務 mail 至學員報名時的電子信箱。\r若需提早或延後開立發票,請發信至hi@aiacademy.tw詢問。發票部分,若有疑慮,請於五日內向校方反應。\n電子發票會於今晚上傳財政部申報後,寄發至你的電子信箱中。

#開學典禮
question: 明天開學典禮我無法參加,需要請假嗎？
answer: 開學典禮及結業典禮皆為課程時間,出缺席將計入缺席時數。若學員因故有請假需求,請填妥請假表單進行申請：https://reurl.cc/xZNOAL,並上傳證明文件,行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。提醒您於下次上課時,攜帶有照片的證件,來領取學員證。

question: 結業相關問題 我當天結業無法出席怎麼辦？
answer: 開學典禮及結業典禮皆為課程時間,出缺席將計入缺席時數。若學員因故有請假需求,請填妥請假表單進行申請：https://reurl.cc/xZNOAL,並上傳證明文件,行政處核可後,系統將會直接寄送請假核可通知至您的信箱中。

#退學
question: 如果我退學了,我還可以登入專區進去看影片等教學相關素材嗎？
answer: 這邊和您說聲抱歉。退學或是終止學員,將無法進入專區觀看相關影片,以維護其他學員的就學權益。

#轉班
question: 開學前申請轉班
answer: 技術班轉經理人班,我們需要檢視您的職級職位作審核。經理人班轉技術班,我們會看您的背景及程式能立,方能決定是否安排入學考試。

question: 開學後申請轉班
answer: 1.除非特殊情況,依學員手冊規定,不許轉班\n2.特殊情況的話(像是個人不可抗拒之因素),需要學員提出證明,學校方得受理 3.課程進行兩周之後,也不許轉班

#補刷
question: 如何補刷
answer: 可使用報到處的平板補刷或是發信至hi@aiacademy.tw /私訊台灣人工智慧學校學務,並提供你的學號/姓名及補刷時間,即可。以上資訊供您參考,若您還有其他疑問也歡迎再與我們聯繫。

#競賽
question: 請問要結業的其中一項是繳交競賽的報告,若團隊中有人幾乎0互動,在現在幾乎是線上上課也只能線上互動的,好像完全沒有約束力
answer: 請組長分配給所有組員該負責的任務,若有組員拒不合作,再請組長私訊告知,我們再去與該學員私下勸說。我們又會有組員互評機制。謝謝

question: 請問每人需參與 AI 產業創新競賽？
answer: 參加AI創新競賽是結業的其中一個門檻。

#颱風
question: 如果遇到颱風天,你們課程會順延嗎？
answer: 如遇颱風天,本校活動、上課是否照常進行,將依行政院人事行政局公布之臺北市停班停課標準為主,相關消息將另行公告於官網、官方臉書。

#學員證
question: 我的學員證不見了,要申請,要怎麼給你們錢?
answer: 您好,請發信至hi@aiacademy.tw 申請,以便我們作業紀錄登記；若您確定申請補發學員證,我們將製作好之後,會現場給您學員證+發票,屆時會與您收500元。

#校友身分
question: 如何核實校友身分?
answer: 具備台灣人工智慧學校結業證書者。

#學號
question: 要如何查詢我的學號?
answer: 請至官網,點選校友資源內的學號查詢即可。

#結業證書
question: 數位結業證書申請補發?
answer: 申請證書,需收取500元申請費用,將開立發票,請提供發票寄件地址。

question: 結業證書寄送問題
answer: 學校將於結業典禮後㇐週內以電子郵件寄發圖靈數位證書至您提供的電子信箱

question: 未收到數位結業證書?
answer: 請至證書入口網站https://global.turingcerts.com/login (使用者登入)使用自己受訓登記的電子郵件註冊進入,即可看到自己證書。第一次登入需要先註冊,電子郵件以報名時的E-mail信箱。註冊後的登入頁面,雖然要求以手機號碼登入,但實質上是使用受訓登記的電子郵件進行登入~  可詳閱收證方使用手冊 https://taiwan-e-portfolio-alliance.gitbook.io/turing-certs-user-guidebook/


#技術領袖全域班 入學考試
question: 何時會寄發技術領袖全域班入學考試連結?
answer: 技術領袖全域班考試連結會於考試前一天17:00前寄發連結並加發簡訊給應考者,線上入學考的連結於考試開始方可登入。

question: 為何我未通過技術領袖全域班入學考試?
answer: 謝謝您的來信。很遺憾您未通過技術領袖全域班入學考試。試題中,選擇題與程式題各佔50分,總分為100分,本期考生的程度都相當好,在參酌其他考生的成績後,我們會擇優錄取。

question: 想報考技術領袖全域班,請問一下 有考古題嗎?
answer: 我們沒有考古題。入學考目前focus在基礎的Python, 所以不管是坊間的參考書,或是學校官方網站的建議內容,應該都可以讓學員有能力通過入學考。謝謝

question: 請問您們技術領袖全域班上課幾周? 總時數為?
answer: 技術領袖全域班上課約十五周,為每周星期三、星期五和星期六,時間為早上 09:30到下午17:00,總計上課時數約 270小時。技術領袖全域班學員缺席達十分之一上課時數者,不予發給結業證書；您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。


#產業專班
question: 為何我未錄取產業專班?
answer: 謝謝您的來信。由於報名實在太熱烈及踴躍、報考者都十分優秀,我們能錄取的考生非常有限,不免會有遺珠之憾,對於產業專班,校方希望提供給在職人士進修與推動產業轉型AI機會, 台灣人工智慧學校經理人班審查標準有二： 1. 從事AI相關工作經理人 2. 具備決策權與預算權的經理人 3. 若以上兩者符合,根據職稱高低以及公司產業別排序 非常的可惜未能錄取所有優秀的報考者,我們期待您未來的參與。建議報考者能就近多利用其他分校,希望及歡迎您報名其他分校或再次報考下一梯次的產業專班。

#專題實作班
question: 請問您們專題實作班上課幾周? 總時數為?
answer: 專題實作班上課五周,為每周星期四和星期五晚間19:00到21:00,星期六時間為早上 09:00到下午17:00,總計上課時數約 46.5小時。您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。

question: 關於【 NLP 專題實作班 】入學考試型式與範圍?
answer: 關於入學考試型式與範圍概述如下：1. 選擇題 10-15 題 ： 機器學習 / 深度學習 / NLP / Python / PyTorch 之基礎概念 2. 手寫題 3 題 ： Python / PyTorch 不要求全對 /  有寫的要正確即可\n以上請參考並保持平常心即可！

question: 請問您們專題實作班上課幾周? 總時數為?
answer: 產業專班上課七周,為每周星期五和星期六,時間為早上 09:00-到下午17:30,總計上課時數為 105小時。您也可以參考官網 https://aiacademy.tw/的招生訊息內的資訊。

#智慧醫療專班結業門檻
question: 結業門檻為何？
answer: 智慧醫療專班學員需參與醫療專班創新競賽且缺席時數不超過15小時(含)即可取得結業證書。產業專班及經理人周末研修班學員需參與 AI 產業創新競賽且缺席時數不超過27小時(含)。

question: 請問您們智慧醫療專班上課幾周? 總時數為?
answer: 智慧醫療專班上課五周,為每星期六,時間為早上 09:30到下午18:00,總計上課時數為 37.5小時。您也可以參考官網 https://aiacademy.tw/的招生訊息內的資訊。

question: 請問智慧醫療專班這門課有要求學員會哪一種程式語言嗎?謝謝
answer: 智慧醫療專班這門課是為了要讓醫學相關領域的專業人士能更快速的掌握人工智慧的知識及資訊,而非專精於程式語言的學習。因此並無特別需要學會哪一種程式語言。相關的課程訊息也歡迎您參考：https://aiacademy.tw/curriculum-med/\n若還有其他的疑問也歡迎與我們聯繫。


#經理人週末研修班
question: 為何我未錄取經理人週末研修班?
answer: 謝謝您的來信。由於報名實在太熱烈及踴躍、報考者都十分優秀,我們能錄取的考生非常有限,不免會有遺珠之憾,對於經理人週末研修班,校方希望提供給在職人士進修與推動產業轉型AI機會, 台灣人工智慧學校經理人班審查標準有二： 1. 從事AI相關工作經理人 2. 具備決策權與預算權的經理人 3. 若以上兩者符合,根據職稱高低以及公司產業別排序 非常的可惜未能錄取所有優秀的報考者,我們期待您未來的參與。建議報考者能就近多利用其他分校,希望及歡迎您報名其他分校或再次報考下一梯次的經理人週末研修班。

question: 經理人週末研修班需要考試或審核嗎？怎麼審核？審核標準是什麼?
answer: 參加經理人週末研修班是不需要經過考試。但需要經過審核,審核的標準為 1. 從事AI相關工作經理人2. 具備決策權與預算權的經理人3. 若以上兩者符合,根據職稱高低排

question: 請問您們經理人週末研修班上課幾週? 總時數為?
answer: 經理人週末研修班上課十四週,為每週星期六,時間為早上 09:00到下午17:30,總計上課時數約 105小時。經理人週末研修班學員缺席達四分之一上課時數者,不予發給結業證書；您也可以參閱官網 https://aiacademy.tw/的招生訊息內的資訊。

